# pyboj Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pyboj/blob/main/examples/quickstart.ipynb)

Python client for the [Bank of Japan Time-Series Statistics API](https://www.stat-search.boj.or.jp/).

Every parameter is typed — no magic strings. The client fetches metadata,
filters series by your criteria, and returns rich domain wrappers with
parsed dates, numeric values, and `to_dataframe()` for pandas.

In [ ]:
!pip install -q pyboj matplotlib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pyboj import (
    BOJ,
    BopAccount,
    Currency,
    Database,
    FofSector,
    Frequency,
    IndexType,
    IndustrySector,
    MonetaryComponent,
    RateCategory,
    TankanIndustry,
    TankanSize,
)

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.3

boj = BOJ()

## 1. Exchange Rates

The most-watched rate in Japan: USD/JPY. Filter by currency pair and frequency.

In [ ]:
rates = boj.exchange_rates(
    currency=Currency.USD_JPY,
    frequency=Frequency.D,
    start_date="202401",
)

r = rates[0]
print(f"{r.currency_pair} | {r.rate_type} | {len(r.dates)} observations")

df = r.to_dataframe()
ax = df.plot(title=f"{r.currency_pair} Daily Rate", legend=False)
ax.set_ylabel("Yen")
plt.tight_layout()
plt.show()

## 2. Interest Rates

Call rates, repo rates, policy rates. Filter by category and collateralization.

In [ ]:
ir = boj.interest_rates(
    category=RateCategory.CALL_RATE,
    frequency=Frequency.D,
    start_date="202401",
)

# Plot all call rate series
dfs = {s.name[:40]: s.to_dataframe()["value"] for s in ir[:4]}
pd.DataFrame(dfs).plot(title="Call Rates")
plt.ylabel("% per annum")
plt.tight_layout()
plt.show()

## 3. TANKAN Survey

Quarterly business sentiment survey of ~10,000 firms. DI > 0 means more firms
report good conditions than bad.

In [ ]:
tankan = boj.tankan(
    industry=TankanIndustry.MANUFACTURING,
    size=TankanSize.LARGE,
    start_date="201501",
)

# Find the Business Conditions DI series
di_series = [t for t in tankan if "business conditions" in (t.name or "").lower()]
if di_series:
    s = di_series[0]
    print(f"{s.name} | {len(s.dates)} observations")
    df = s.to_dataframe()
    ax = df.plot(title="Tankan: Large Manufacturers DI", legend=False)
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="--")
    ax.set_ylabel("DI (% points)")
    plt.tight_layout()
    plt.show()

## 4. Price Indices

Corporate goods prices — a leading indicator for consumer inflation.

In [ ]:
indices = boj.price_indices(
    index_type=IndexType.PRODUCER,
    start_date="202001",
)

if indices:
    s = indices[0]
    print(f"{s.name} | base={s.base_year} | {len(s.dates)} obs")
    df = s.to_dataframe()
    ax = df.plot(title=f"Producer Price Index ({s.base_year}=100)", legend=False)
    ax.set_ylabel("Index")
    plt.tight_layout()
    plt.show()

## 5. Balance of Payments

Japan's current account balance — net trade, income, and services.

In [ ]:
bop = boj.balance_of_payments(
    account=BopAccount.CURRENT,
    frequency=Frequency.M,
    start_date="202001",
)

if bop:
    s = bop[0]
    print(f"{s.name} | {s.unit} | {len(s.dates)} obs")
    df = s.to_dataframe()
    ax = df.plot(title="Current Account Balance", legend=False)
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="--")
    ax.set_ylabel(s.unit or "")
    plt.tight_layout()
    plt.show()

## 6. Monetary Base

Total money supplied by the BOJ. Tracks the magnitude of quantitative easing.

In [ ]:
money = boj.money_deposits(
    component=MonetaryComponent.TOTAL,
    db=Database.MONETARY_BASE,
    start_date="200501",
)

if money:
    s = money[0]
    print(f"{s.name} | {s.unit} | {len(s.dates)} obs")
    df = s.to_dataframe()
    ax = df.plot(title="Monetary Base", legend=False)
    ax.set_ylabel(s.unit or "")
    plt.tight_layout()
    plt.show()

## 7. Loans by Sector

Bank lending to different industry sectors.

In [ ]:
loans = boj.loans(
    sector=IndustrySector.MANUFACTURING,
    start_date="202001",
)

if loans:
    s = loans[0]
    print(f"{s.name} | {s.unit} | {len(s.dates)} obs")
    df = s.to_dataframe()
    ax = df.plot(title="Loans to Manufacturing", legend=False)
    ax.set_ylabel(s.unit or "")
    plt.tight_layout()
    plt.show()

## 8. Flow of Funds

Household financial assets — how much wealth Japanese households hold.

In [ ]:
fof = boj.flow_of_funds(
    sector=FofSector.HOUSEHOLDS,
    start_date="200501",
)

if fof:
    s = fof[0]
    print(f"{s.name} | {s.unit} | {len(s.dates)} obs")
    df = s.to_dataframe()
    ax = df.plot(title="Household Financial Assets", legend=False)
    ax.set_ylabel(s.unit or "")
    plt.tight_layout()
    plt.show()

## More Categories

`BOJ` covers all 13 BOJ categories (43 databases). Here are a few more:

In [ ]:
from pyboj import (
    InstitutionType,
    MarketSegment,
    OperationType,
    StatCategory,
)

# Financial markets
fm = boj.financial_markets(segment=MarketSegment.GOVT_BONDS, db=Database.GOVT_BOND_TRADING)
print(f"Financial markets: {len(fm)} series")

# Balance sheets
bs = boj.balance_sheets(institution_type=InstitutionType.BOJ)
print(f"Balance sheets: {len(bs)} series")

# BOJ operations
ops = boj.boj_operations(operation_type=OperationType.JGB_OPERATIONS)
print(f"BOJ operations: {len(ops)} series")

# Public finance
pf = boj.public_finance()
print(f"Public finance: {len(pf)} series")

# International statistics
intl = boj.international(stat_category=StatCategory.DERIVATIVES, db=Database.DERIVATIVES_MARKET)
print(f"International: {len(intl)} series")

## Discovery: Layer Tree & Search

Don't know what series are available? Use `layer_tree()` to browse the hierarchy
or `search()` to find series by keyword.

In [ ]:
# Browse the exchange rates hierarchy
tree = boj.layer_tree(Database.EXCHANGE_RATES)
print(f"Root has {len(tree.children)} top-level categories:\n")
for child in tree.children:
    print(f"  {child.name} ({len(child.series_codes)} series, {len(child.children)} sub)")

In [ ]:
# Search by keyword
results = boj.search(Database.EXCHANGE_RATES, "dollar")
print(f"Found {len(results)} series matching 'dollar':\n")
for rec in results[:5]:
    print(f"  {rec.SERIES_CODE} | {rec.NAME_OF_TIME_SERIES}")

boj.close()